# OvR

This notebook tries to create a baseline for the task. Noteworthy points:

-Using LRAP loss metric in cross-validation in Model 2. 

-Best score is around 0.3 in Model 2. It seems low but I feel this task is quite difficult so maybe very high scores are difficult.

-Balancing cross-validation using n_jobs and pre_dispatch in order to preserve maximum CPU usage but prevent memory from blowing up.

In [1]:
import pandas as pd
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.metrics import label_ranking_average_precision_score,make_scorer
import random

# Download data

In [2]:
#Load the previously transformed data.
NUM_FEATURES = 5000
NUM_CLASSES = 3993

x_train = pd.read_csv("../data/expanded/train_features.csv", names=range(NUM_FEATURES))
y_train = pd.read_csv("../data/expanded/train_labels.csv", names=range(NUM_CLASSES))

x_val = pd.read_csv("../data/expanded/dev_features.csv", names=range(NUM_FEATURES))
y_val = pd.read_csv("../data/expanded/dev_labels.csv", names=range(NUM_CLASSES))

# Model 1

In [ ]:
#Training takes around 30min on my machine. 5000 classes  = 5000 seperate models in one-vs-rest scheme so not too efficient.
model = OneVsRestClassifier(LinearSVC(tol = np.exp(-4), random_state=0, dual = False),n_jobs = -1).fit(x_train,y_train)

In [ ]:
y_pred = model.predict(x_val)
np.savetxt("../public data/svm.csv", y_pred, delimiter=",")

In [ ]:
score = label_ranking_average_precision_score(y_val,y_pred)
score

# Model 2 : Pegasos

In [ ]:

def pegasos_fast(x_train,y_train, max_epoch, lam, watch_list=None, grad_checking=False, tfidf= False,min_loss = 0.01):
    """
    Description
    ==========
    Implementation of Pegasos Algorithm 

    Input
    =====
    review_list: list of reviewInstance's
    list of objects with labels and encoded input from reviews

    max_epoch: int 
    stopping condition

    lam: float 
    regularization parameter

    watch_list: list or reviewInstance's
    passed to accuracy_percent or magnitude_compare; default None

    grad_checking: bool 
    numerical test of gradient of svm objective
    
    tfidf: bool
    use tf-idf encoding of text in review_list

    Output
    ======
    weights
    """

    #Initialization
    w = np.zeros(x_train.shape[1])
    epoch = 0
    t = 1.
    samples = list(range(len(x_train)))
    s = 1.

    #Loop
    # Use the util.increment and util.dotProduct functions in update
    #while training_loss > min_loss:
    while epoch <max_epoch:
          
        random.shuffle(samples)
        for i in range(len(x_train)):
            t +=1
            lr = 1 / (t*lam)
            margin = y_train[i]*s*np.dot(w,x_train[i,:])

            scale = 1-lr*lam
            s = s*scale
            if s == 0:
                s = 1
                w = np.zeros(x_train.shape[1])
                
            if margin <1:
                second_scale = (1/s)*lr*y_train[i]
                w = w + x_train[i,:]*second_scale

        epoch +=1
        #temp_W = np.copy(w)
        #temp_W = temp_W*s
        #training_loss = pegasos_loss(x_train,y_train,temp_W)
        #print(training_loss)    
    w = w*s
    return w

In [ ]:
y_single= y_train.values[:,3]
y_single[y_single==0] =-1

x_single = x_train.values

#pegasos_fast(x_single,y_single,4,0.1)

In [ ]:
%prun pegasos_fast(x_single,y_single,4,0.1)

In [ ]:
def pegasos_loss(x,y,weight):
    loss = 0
    for i in range(len(x)):     
        prediction = svm_predict(x[i,:],weight)
        
        if prediction != y[i]:
            loss+=1
    return loss/len(x)

In [ ]:
def svm_predict(review_X, weight):
    if np.dot(review_X,weight)>0:
        return 1
    else:
        return -1

# Training Pegasos - multiclass

In [ ]:
y_train_temp = y_train.values#[:,:1000]
x_train_temp = x_train.values

In [ ]:
w = np.zeros((x_train.shape[1],y_train.values.shape[1]))

for i in range(y_train_temp.shape[1]):
    if i % 50==0:
        print(i)
    temp_w = pegasos_fast(x_train_temp,y_train_temp[:,i],5,0.1)
    w[:,i]=temp_w

In [ ]:
def multi_class_predict(x,w):
    pred = np.matmul(x,w)
    return pred

In [ ]:
y_pred = multi_class_predict(x_val.values,w)

In [ ]:
score = label_ranking_average_precision_score(y_val.values[:,:y_train.values.shape[1]],y_pred)
score

In [ ]:
y_train.values.shape[1]

# RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_samples = 5000,n_jobs=1)

In [ ]:
rf.fit(x_train,y_train)

In [ ]:
rf.predict(x_val)

# KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=10, metric = 'hamming')

In [23]:
neigh.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='hamming',
                     metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                     weights='uniform')

In [24]:
y_pred = neigh.predict(x_val)

In [26]:
neigbhours = [3,5,10,20]
weights = ['uniform','distance']
pred_scores = []

for n in neigbhours:
    for w in weights:
        neigh = KNeighborsClassifier(n_neighbors=n,weights = w, metric = 'hamming')
        neigh.fit(x_train,y_train)
        y_pred = neigh.predict(x_val)
        score = label_ranking_average_precision_score(y_val,y_pred)
        pred_scores.append(score)

In [27]:
pred_scores

[0.19741457592804107,
 0.21043060752771908,
 0.1649265464700003,
 0.18712780713355143,
 0.11333454455597061,
 0.15900607232951927,
 0.07824598601447114,
 0.11789113187898997]

# best is 5 and uniform

In [36]:
neigh = KNeighborsClassifier(n_neighbors=3, metric = 'hamming')

In [ ]:
neigh.fit(x_train,y_train)
y_pred = neigh.predict(x_val)

In [35]:
score = label_ranking_average_precision_score(y_val,y_pred)
score

0.1649265464700003

In [30]:
y_pred.shape

(1314, 3993)

In [31]:
y_val.shape

(1314, 3993)

In [32]:
y_pred

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [34]:
np.savetxt("../public data/knn.csv", y_pred, delimiter=",")